Joseph Okere, Toronto Neighborhoods

In [2]:
# Import all classes needed

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

!conda install -c conda-forge urllib3 --yes
import urllib3

from lxml import html
from bs4 import BeautifulSoup
import requests
import certifi
from geopy import point as gPoint

print('All Libraries imported.')


Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

Here, we grab the link from the lab, and use BeautifulSoup to get the data.

In [3]:
http = urllib3.PoolManager(maxsize=10,
                           cert_reqs='CERT_REQUIRED',
                           ca_certs=certifi.where())
quote_page = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = http.request('GET', quote_page)
soup = BeautifulSoup(response.data)

In the HTML document, the header is within <th> rows, while the data sits inside <tr> rows. I get both, and then create the dataframe.

In [4]:
wholeTable  = soup.find("table", attrs={"class": "wikitable sortable"})
t_row  = wholeTable.find("tr")
t_row

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>

In [5]:
columnNames = []
t_rowHeaders = t_row.find("th").string #get first one
print(t_rowHeaders)
columnNames.append(t_rowHeaders)

while((t_rowHeaders.find_next("th").string) != None):
    t_rowHeaders = t_rowHeaders.find_next("th").string
    print(t_rowHeaders)
    columnNames.append(t_rowHeaders)
    
columnNames

Postcode
Borough
Neighbourhood



['Postcode', 'Borough', 'Neighbourhood\n']

In [6]:
fsa_Array = []
while(t_row.find_next("tr") != None):
    t_row = t_row.find_next("tr")  
    
    fsa_ArrLine = []
    
    t_Elements = t_row
    for t in t_Elements:
        if("a href" in str(t)):
            fsa_ArrLine.append(t.find("a").string.replace("\n", ""))
        else:
            fsa_ArrLine.append(str(t.string).replace("\n",""))
    
    fsa_Array.append(fsa_ArrLine)

fsa_Array

[['', 'M1A', '', 'Not assigned', '', 'Not assigned'],
 ['', 'M2A', '', 'Not assigned', '', 'Not assigned'],
 ['', 'M3A', '', 'North York', '', 'Parkwoods'],
 ['', 'M4A', '', 'North York', '', 'Victoria Village'],
 ['', 'M5A', '', 'Downtown Toronto', '', 'Harbourfront'],
 ['', 'M5A', '', 'Downtown Toronto', '', 'Regent Park'],
 ['', 'M6A', '', 'North York', '', 'Lawrence Heights'],
 ['', 'M6A', '', 'North York', '', 'Lawrence Manor'],
 ['', 'M7A', '', "Queen's Park", '', 'Not assigned'],
 ['', 'M8A', '', 'Not assigned', '', 'Not assigned'],
 ['', 'M9A', '', 'Etobicoke', '', 'None'],
 ['', 'M1B', '', 'Scarborough', '', 'Rouge'],
 ['', 'M1B', '', 'Scarborough', '', 'Malvern'],
 ['', 'M2B', '', 'Not assigned', '', 'Not assigned'],
 ['', 'M3B', '', 'North York', '', 'Don Mills North'],
 ['', 'M4B', '', 'East York', '', 'None'],
 ['', 'M4B', '', 'East York', '', 'None'],
 ['', 'M5B', '', 'Downtown Toronto', '', 'Ryerson'],
 ['', 'M5B', '', 'Downtown Toronto', '', 'Garden District'],
 ['', 'M

In [7]:
tempDF = pd.DataFrame(fsa_Array)
tempDF

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,,M1A,,Not assigned,,Not assigned,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,,M2A,,Not assigned,,Not assigned,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,,M3A,,North York,,Parkwoods,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,,M4A,,North York,,Victoria Village,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,,M5A,,Downtown Toronto,,Harbourfront,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,,M5A,,Downtown Toronto,,Regent Park,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,,M6A,,North York,,Lawrence Heights,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,,M6A,,North York,,Lawrence Manor,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,,M7A,,Queen's Park,,Not assigned,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,,M8A,,Not assigned,,Not assigned,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


From here on out, i get rid of all of the unnecessary information and format the table for the first part of the lab.

In [8]:
toronto_DF = tempDF[[1,3,5]]
toronto_DF.columns = columnNames
toronto_DF = toronto_DF.rename(index=str, columns={"Postcode":"PostalCode", "Neighbourhood\n":"Neighborhood"})
toronto_DF

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [9]:
t_DF = toronto_DF[toronto_DF.Borough != "Not assigned"]
t_DF

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,None
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [10]:
t_DF2 = t_DF.loc[t_DF["Neighborhood"] != "None"]
t_DF2.reset_index(inplace=True)
t_DF2

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M5A,Downtown Toronto,Regent Park
4,6,M6A,North York,Lawrence Heights
5,7,M6A,North York,Lawrence Manor
6,8,M7A,Queen's Park,Not assigned
7,11,M1B,Scarborough,Rouge
8,12,M1B,Scarborough,Malvern
9,14,M3B,North York,Don Mills North


In [11]:
t_DF3 = t_DF2.drop([177,178,179,180])
t_DF3

,index,PostalCode,Borough,Neighborhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M5A,Downtown Toronto,Regent Park
4,6,M6A,North York,Lawrence Heights
5,7,M6A,North York,Lawrence Manor
6,8,M7A,Queen's Park,Not assigned
7,11,M1B,Scarborough,Rouge
8,12,M1B,Scarborough,Malvern
9,14,M3B,North York,Don Mills North


I needed to try and get all values in "Borough" where "Neighborhood" was equal to "Not assigned". The "Series.base" is depreciated, but it worked for my purposes here. I was able to first look to see that it worked, and then I applied it to the entire dataframe.

In [12]:
t_DF4 = t_DF3
look = t_DF4[t_DF4["Neighborhood"] == "Not assigned"].Borough.base[2][0]
look

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Series.base is deprecated and will be removed in a future version
  


"Queen's Park"

In [13]:
t_DF5 = t_DF4
t_DF6 = t_DF5.replace("Not assigned", t_DF4[t_DF4["Neighborhood"] == "Not assigned"].Borough.base[2][0])
t_DF6.drop("index", axis=1, inplace=True)
t_DF6

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Series.base is deprecated and will be removed in a future version
  


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


I didn't really know how to do this by within the dataframe itself, so I hope that I can look back on this one day and say "you were such a novice". Essentially, here I read each record out of the dataframe and construct a multidimensional array, where all neighborhoods are grouped by Postal codes. The results fit the specs of the lab instructions.

In [14]:
t_DF7 = t_DF6
testarr1 = []
postalVal = ""
boroughVal = ""
neighVal = ""
for index, asdf in t_DF7.iterrows():
    if(index == 0):
        postalVal = asdf.PostalCode
        boroughVal = asdf.Borough
        neighVal = asdf.Neighborhood
        df = [postalVal, boroughVal, neighVal]
    if(index > 0):
        if(postalVal != asdf.PostalCode):
            df = [postalVal, boroughVal, neighVal]
            testarr1.append(df)
            postalVal = asdf.PostalCode
            boroughVal = asdf.Borough
            neighVal = asdf.Neighborhood
        else:
            neighVal = neighVal + ", " + asdf.Neighborhood
            
testarr1

[['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront, Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights, Lawrence Manor'],
 ['M7A', "Queen's Park", "Queen's Park"],
 ['M1B', 'Scarborough', 'Rouge, Malvern'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M5B', 'Downtown Toronto', 'Ryerson, Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M9B', 'Etobicoke', 'Cloverdale, Martin Grove, Princess Gardens'],
 ['M1C', 'Scarborough', 'Highland Creek, Port Union'],
 ['M3C', 'North York', 'Flemingdon Park, Don Mills South'],
 ['M5C', 'Downtown Toronto', 'St. James Town'],
 ['M9C',
  'Etobicoke',
  'Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe'],
 ['M1E', 'Scarborough', 'Guildwood, Morningside, West Hill'],
 ['M4E', 'East Toronto', 'The Beaches'],
 ['M5E', 'Downtown Toronto', 'Berczy Park'],
 ['M6E', 'York', 'Caledonia-Fairbanks'],
 ['M1G', 'Scarborough', 'Woburn'],
 ['M4G', 'East York', 'Leasid

In [15]:
grouped_toronto_DF = pd.DataFrame(testarr1)
grouped_toronto_DF.columns = columnNames
grouped_toronto_DF = grouped_toronto_DF.rename(index=str, columns={"Postcode":"PostalCode", "Neighbourhood\n":"Neighborhood"})
grouped_toronto_DF

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M1B,Scarborough,"Rouge, Malvern"
6,M3B,North York,Don Mills North
7,M5B,Downtown Toronto,"Ryerson, Garden District"
8,M6B,North York,Glencairn
9,M9B,Etobicoke,"Cloverdale, Martin Grove, Princess Gardens"


In [16]:
grouped_toronto_DF.shape

(95, 3)

Here is the begininning of part 2's lab. 

In [17]:
!wget -q -O 'Geospacial_data.csv' http://cocl.us/Geospatial_data
latlngDF = pd.read_csv("Geospacial_data.csv")
latlngDF

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [18]:
latlngDF = latlngDF.rename(index=str, columns={"Postal Code":"PostalCode"})
latlngDF

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [19]:
df_a = grouped_toronto_DF
df_b = latlngDF
final_DF = df_a.join(df_b.set_index("PostalCode"), on="PostalCode")
final_DF

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
6,M3B,North York,Don Mills North,43.745906,-79.352188
7,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
8,M6B,North York,Glencairn,43.709577,-79.445073
9,M9B,Etobicoke,"Cloverdale, Martin Grove, Princess Gardens",43.650943,-79.554724


In [25]:
final_DF.shape

(95, 5)

In [30]:
toronto_address = 'Toronto, CA'

toronto_geolocator  = Nominatim(user_agent="toronto_explorer")
toronto_location    = toronto_geolocator.geocode(toronto_address)
toronto_latitude    = toronto_location.latitude
toronto_longitude   = toronto_location.longitude

map_Toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(final_DF['Latitude'], final_DF['Longitude'], final_DF['Borough'], final_DF['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

I decided that I'll just want to search neighborhoods in the Scarborough borough.

In [31]:
scarborough_DF = final_DF[final_DF['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_DF.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


From here, i'm following what was done in the ungraded lab for New York.

In [33]:
sc_address = 'Scarborough, CA'

sc_geolocator = Nominatim(user_agent="toronto_explorer")
sc_location   = sc_geolocator.geocode(sc_address)
sc_latitude   = sc_location.latitude
sc_longitude  = sc_location.longitude
print('The geograpical coordinate of Scarborough, Toronto, Canada are {}, {}.'.format(sc_latitude, sc_longitude))

The geograpical coordinate of Scarborough, Toronto, Canada are 43.773077, -79.257774.


In [34]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[sc_latitude, sc_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_DF['Latitude'], scarborough_DF['Longitude'], scarborough_DF['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

In [35]:
CLIENT_ID = 'QSCI5VKZBNWNFDNBFQZI5ABAWLQ3L1USCPJ4HWPKX3XZLY24' # your Foursquare ID
CLIENT_SECRET = 'TJOIVHVVTG1WPSJMEE42GBHBFRNT21HVAVOALW0TKU2DQNYZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QSCI5VKZBNWNFDNBFQZI5ABAWLQ3L1USCPJ4HWPKX3XZLY24
CLIENT_SECRET:TJOIVHVVTG1WPSJMEE42GBHBFRNT21HVAVOALW0TKU2DQNYZ


In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
scarborough_venues = getNearbyVenues(names=scarborough_DF['Neighborhood'],
                                     latitudes=scarborough_DF['Latitude'],
                                     longitudes=scarborough_DF['Longitude']
                                     )

Rouge, Malvern
Highland Creek, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West


In [41]:
print(scarborough_venues.shape)
scarborough_venues.head()

(92, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Highland Creek, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Highland Creek, Port Union",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


Seeing how many venues were returned for each neighborhood, like the NY lab:

In [42]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,6,6,6,6,6,6
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Clarks Corners, Sullivan, Tam O'Shanter",11,11,11,11,11,11
"Cliffcrest, Cliffside, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Scarborough Town Centre",7,7,7,7,7,7
"East Birchmount Park, Ionview",4,4,4,4,4,4
"Guildwood, Morningside, West Hill",8,8,8,8,8,8


..and also, all unique categories:

In [43]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 54 uniques categories.


Analyze each neighborhood.

In [44]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Port Union",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Port Union",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [45]:
#get size
scarborough_onehot.shape

(92, 55)

Here, we'll group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [46]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Grocery Store,Hakka Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.0,0.00,0.000,0.166667,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000,0.000000,0.00,0.000000,0.166667,0.000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000,0.166667,0.000000,0.166667,0.0,0.000,0.000000,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.25,0.000,0.000000,0.000000,0.25,0.000000,0.00,0.00,0.00,0.000,0.000000,0.000000,0.000000,0.25,0.000000,0.000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.250000,0.0,0.000,0.000000,0.000000,0.000000
2,Cedarbrae,0.000000,0.125,0.000000,0.125000,0.125000,0.000000,0.000000,0.0,0.0,0.00,0.125,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000,0.000000,0.125000,0.000000,0.00,0.000000,0.125,0.000000,0.000,0.000000,0.00,0.000000,0.125000,0.000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000,0.125000,0.000000,0.000000
3,"Clairlea, Golden Mile, Oakridge",0.000000,0.000,0.000000,0.200000,0.000000,0.000000,0.000000,0.2,0.1,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000,0.100000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.100,0.000000,0.00,0.000000,0.000000,0.000,0.1,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.1,0.000,0.000000,0.000000,0.000000
4,"Clarks Corners, Sullivan, Tam O'Shanter",0.000000,0.000,0.000000,0.000000,0.090909,0.000000,0.000000,0.0,0.0,0.00,0.000,0.090909,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000,0.090909,0.090909,0.000000,0.00,0.000000,0.000,0.000000,0.000,0.090909,0.00,0.000000,0.000000,0.000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.090909,0.181818,0.000000,0.000000,0.000,0.000000,0.090909,0.000000,0.0,0.000,0.090909,0.000000,0.000000
5,"Cliffcrest, Cliffside, Scarborough Village West",0.333333,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000,0.0,0.000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000
6,"Dorset Park, Scarborough Town Centre",0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.000,0.142857,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000,0.000000,0.000000,0.142857,0.00,0.000000,0.000,0.285714,0.000,0.000000,0.00,0.142857,0.000000,0.000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.142857
7,"East Birchmount Park, Ionview",0.000000,0.000,0.000000,0.000000,0.000000

Let's print each neighborhood along with the top 5 most common venues

In [47]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0              Lounge  0.17
1      Breakfast Spot  0.17
2        Skating Rink  0.17
3      Sandwich Place  0.17
4  Chinese Restaurant  0.17


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1  General Entertainment  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                venue  freq
0              Lounge  0.12
1              Bakery  0.12
2                Bank  0.12
3     Thai Restaurant  0.12
4  Athletics & Sports  0.12


----Clairlea, Golden Mile, Oakridge----
           venue  freq
0         Bakery   0.2
1       Bus Line   0.2
2  Metro Station   0.1
3   Intersection   0.1
4   Soccer Field   0.1


----Clarks Corners, Sullivan, Tam O'Shanter----
                  venue  freq
0           Pizza Place  0.18
1         Shopping Mall  0.09
2   Fried Chicken Joint  0.09
3  Fast Food Restaurant  0.09
4          Noodle Hous

Put it all into a pandas dataframe.

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Chinese Restaurant,Skating Rink,Sandwich Place,Print Shop,Convenience Store,Department Store,Discount Store,Electronics Store
1,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Grocery Store,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
2,Cedarbrae,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Lounge,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store
3,"Clairlea, Golden Mile, Oakridge",Bus Line,Bakery,Intersection,Fast Food Restaurant,Metro Station,Bus Station,Park,Soccer Field,Bar,Convenience Store
4,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Noodle House,Shopping Mall,Pharmacy,Fast Food Restaurant,Bank,Chinese Restaurant,Italian Restaurant,Fried Chicken Joint,Thai Restaurant


Cluster neighborhoods, using Kmeans.

In [50]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 4, 0, 0, 0, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_DF

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2,Fast Food Restaurant,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Electronics Store,Discount Store
1,M1C,Scarborough,"Highland Creek, Port Union",43.784535,-79.160497,3,Bar,Moving Target,Construction & Landscaping,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Intersection,Spa,Electronics Store,Rental Car Location,Breakfast Spot,Medical Center,Pizza Place,Mexican Restaurant,College Stadium,Furniture / Home Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Convenience Store,College Stadium,Grocery Store,General Entertainment,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Lounge,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store


Now, to map clusters.

In [52]:
# create map
map_clusters = folium.Map(location=[sc_latitude, sc_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters